In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA


# Load Processed Tesla Data

In [ ]:
tsla = pd.read_csv(
    "data/processed/TSLA_processed.csv",
    index_col=0,
    parse_dates=True
)

tsla.head()


# Use Full Historical Data for Final Model

In [ ]:
tsla_close = tsla["Adj Close"]


# Train Final ARIMA Model

In [ ]:
final_model = ARIMA(tsla_close, order=(5, 1, 0))
final_model_fit = final_model.fit()

final_model_fit.summary()


# Forecast Horizon (12 Months)

In [ ]:
FORECAST_DAYS = 252

forecast_result = final_model_fit.get_forecast(steps=FORECAST_DAYS)

forecast_mean = forecast_result.predicted_mean
forecast_ci = forecast_result.conf_int()


# Create Forecast Date Index

In [ ]:
last_date = tsla_close.index[-1]
forecast_index = pd.bdate_range(
    start=last_date,
    periods=FORECAST_DAYS + 1,
    closed="right"
)

forecast_mean.index = forecast_index
forecast_ci.index = forecast_index


# Plot Forecast with Confidence Intervals

In [ ]:
plt.figure(figsize=(14, 7))

plt.plot(tsla_close.index, tsla_close, label="Historical Data")
plt.plot(forecast_mean.index, forecast_mean, label="Forecast", color="orange")

plt.fill_between(
    forecast_ci.index,
    forecast_ci.iloc[:, 0],
    forecast_ci.iloc[:, 1],
    color="orange",
    alpha=0.3,
    label="Confidence Interval"
)

plt.title("Tesla Stock Price Forecast (12 Months)")
plt.xlabel("Date")
plt.ylabel("Adjusted Close Price")
plt.legend()
plt.show()


# Forecast Trend Analysis

In [ ]:
forecast_return = (forecast_mean.iloc[-1] - tsla_close.iloc[-1]) / tsla_close.iloc[-1]

print(f"Expected 12-month return: {forecast_return:.2%}")


# Confidence Interval Width Analysis

In [ ]:
ci_width = forecast_ci.iloc[:, 1] - forecast_ci.iloc[:, 0]

plt.figure(figsize=(12, 5))
plt.plot(ci_width)
plt.title("Forecast Uncertainty Over Time")
plt.xlabel("Date")
plt.ylabel("Confidence Interval Width")
plt.show()


# Opportunities & Risks Summary (Code Support)

In [ ]:
print("Opportunities:")
print("- Potential price appreciation based on forecast trend")
print("- Momentum continuation if macro conditions remain favorable\n")

print("Risks:")
print("- Wide confidence intervals indicate high uncertainty")
print("- External shocks not captured by historical data")


# Save Forecast Output

In [ ]:
forecast_df = pd.DataFrame({
    "Forecast": forecast_mean,
    "Lower_CI": forecast_ci.iloc[:, 0],
    "Upper_CI": forecast_ci.iloc[:, 1]
})

forecast_df.to_csv("data/processed/TSLA_forecast.csv")
